In [ ]:
# 作成した学習器モデルを用いて、予測を行う
#2-3keVと3-10keVで分類を行い、どちらの学修器でもFlare判定されたデータを出力する
#

# import

In [1]:
from keras import backend as K
import tensorflow as tf

K.clear_session()
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)
K.set_session(sess)

from keras.models import load_model
import os
from keras.preprocessing.image import img_to_array, load_img
from keras.models import load_model
import numpy as np
from PIL import Image
import pandas as pd
pd.set_option('display.max_rows', 1000)



Using TensorFlow backend.


# load model

In [2]:
#学習済みモデルの読込
model_3_10keV_name='./3-10keV_5000/model_.05-0.04.hdf5'
model_3_10keV=load_model(model_3_10keV_name)
model_2_3keV_name = './2-3keV_5000/model_.09-0.07.hdf5'
model_2_3keV=load_model(model_2_3keV_name)



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 520)       5200      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 864)       4044384   
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 928)       7217056   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 928)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 928)       0         
_______________________________________

# load TestImage

In [5]:
#推測するディレクトリーを指定

path_2_3keV='/home/tsato/work/machine_learning/CNN/images/1orbit/2-3keV/test/'
path_3_10keV='/home/tsato/work/machine_learning/CNN/images/1orbit/3-10keV/test/'

#path_2_3keV='/home/tsato/work/MAXI/HR1099/ML_noDetection/58209/4orbit/image/2-3'
#path_3_10keV='/home/tsato/work/MAXI/HR1099/ML_noDetection/58209/4orbit/image/3-10'

#path_2_3keV='/home/tsato/Dropbox/Public/MAXI_flare_ana/VYAri_20200221/yuido/ML/2-3'
#path_3_10keV='/home/tsato/Dropbox/Public/MAXI_flare_ana/VYAri_20200221/yuido/ML/3-10'


#2-3keV
print('target name:')
print(os.listdir(path_2_3keV))
TARGET_DIR = input('>>> ')
print('predict directory:')
print(os.listdir(path_2_3keV +'/'+ TARGET_DIR))
IMAGE_DIR = input('>>> ')

images_2_3keV = os.listdir(path_2_3keV + '/' +TARGET_DIR+'/'+IMAGE_DIR)

#3-10keV
images_3_10keV = os.listdir(path_3_10keV + '/' +TARGET_DIR +'/'+IMAGE_DIR)


#print("2-3keV:" + str(len(images_2_3keV)))
#print("3-10keV:" + str(len(images_3_10keV)))

target name:
['correction', 'cdfS', 'RXTE6', 'YZCMi', 'ARPsc', 'V1055Ori', 'UXAri', 'SZPsc', 'IMPeg', 'lamand', 'CFTuc', 'ARLac', 'IIPeg', 'HR5110', 'GTMus', 'CygX-3', 'SigmaGem', 'V824Ara', 'Algol', 'V841Cen', 'VYAri', 'HR1099', 'RXTE2']
>>> RXTE2
predict directory:
['2013', '2017', '2009', 'ALL', '2018', '2015', '2012', '2016', '2010', '2014', '2011']
>>> ALL


# Predict

In [7]:
#2-3keV
df = []
for jpeg_name in images_2_3keV:
    img_path = (jpeg_name)
    img = img_to_array(load_img(path_2_3keV + TARGET_DIR +'/'+ IMAGE_DIR + '/' + img_path, target_size=(32,32)).convert('L'))
    #img = img_to_array(load_img(path_2_3keV +'/'+ IMAGE_DIR + '/' + img_path, target_size=(32,32)).convert('L'))
    img_nad = img_to_array(img)/255	    #0-1に変換
    img_nad = img_nad[None, ...]	    #四次元配列に
    label=['background','flare']
    pred = model_2_3keV.predict(img_nad, batch_size=1, verbose=0)
    score = np.max(pred)
    pred_label = label[np.argmax(pred[0])]
    val = 0.99
    if score > val and pred_label == "flare":
        a=jpeg_name.split("_")
        list=[int(a[0]),jpeg_name,score]     
        df.append(list)
result_2_3keV = pd.DataFrame(df)
result_2_3keV = result_2_3keV.sort_values([0])
result_2_3keV = result_2_3keV.reset_index(drop=True)
result_2_3keV = result_2_3keV.iloc[:,0:3]
#print(result)

#3-10keV
df = []
for jpeg_name in images_3_10keV:
    img_path = (jpeg_name)
    img = img_to_array(load_img(path_3_10keV + TARGET_DIR +'/'+ IMAGE_DIR + '/' + img_path, target_size=(32,32)).convert('L'))
    #img = img_to_array(load_img(path_3_10keV + '/' +  IMAGE_DIR + '/' + img_path, target_size=(32,32)).convert('L'))
    img_nad = img_to_array(img)/255	    #0-1に変換
    img_nad = img_nad[None, ...]	    #四次元配列に
    label=['background','flare']
    pred = model_3_10keV.predict(img_nad, batch_size=1, verbose=0)
    score = np.max(pred)
    pred_label = label[np.argmax(pred[0])]
    val = 0.99
    if score > val and pred_label == "flare":
        a=jpeg_name.split("_")
        list=[int(a[0]),jpeg_name,score]     
        df.append(list)
result_3_10keV = pd.DataFrame(df)
result_3_10keV = result_3_10keV.sort_values([0])
result_3_10keV = result_3_10keV.reset_index(drop=True)
result_3_10keV = result_3_10keV.iloc[:,0:3]
#print(result)

In [14]:
#2-3keVと3-10keVの重複したデータのみを抽出
result = result_2_3keV[result_2_3keV[1].isin(result_3_10keV[1])]
result = result.reset_index(drop=True)
result=result.iloc[:,1:2]
result.index = result.index +1

#データ数を表示
result

,1
1,322_55066.218125-55066.222396.jpg
2,861_55087.187870-55087.190891.jpg
3,1247_55121.254722-55121.257604.jpg
4,1635_55707.909144-55707.910289.jpg
5,2415_56498.747801-56498.750382.jpg
6,3440_55853.336458-55853.338391.jpg


In [9]:
result_2_3keV[1]

0       037_55055.170972-55055.174317.jpg
1       050_57767.479560-57767.482535.jpg
2       051_55201.066713-55201.070683.jpg
3       056_55201.385208-55201.387836.jpg
4       057_55566.037269-55566.039734.jpg
5       057_58129.173773-58129.176678.jpg
6       065_55202.145069-55202.149155.jpg
7       067_55056.696227-55056.699757.jpg
8       070_55202.462963-55202.466470.jpg
9       070_55056.823380-55056.826887.jpg
10      078_55057.253565-55057.257049.jpg
11      080_55057.317130-55057.320567.jpg
12      082_55057.380694-55057.384178.jpg
13      088_55057.571366-55057.574919.jpg
14      092_55568.644190-55568.647118.jpg
15      096_55057.825602-55057.829178.jpg
16      100_56302.755324-56302.757743.jpg
17      109_55205.000069-55205.004363.jpg
18      127_55059.492963-55059.496840.jpg
19      143_56686.769398-56686.772187.jpg
20      144_58135.349051-58135.351910.jpg
21      148_57774.428056-57774.430914.jpg
22      156_57775.007037-57775.009873.jpg
23      163_57052.764375-57052.767

In [43]:
#2020/03/05
#3-10keVだけで調査

#Load Image
path_3_10keV='/home/tsato/work/mxondem/RXTE2/2018/3-10/crop/'
images_3_10keV = os.listdir(path_3_10keV)

#Predicrt
df = []
for jpeg_name in images_3_10keV:
    img_path = (jpeg_name)
    #img = img_to_array(load_img(path_3_10keV + TARGET_DIR +'/'+ IMAGE_DIR + '/' + img_path, target_size=(32,32)).convert('L'))
    img = img_to_array(load_img(path_3_10keV + '/' + img_path, target_size=(32,32)).convert('L'))
    img_nad = img_to_array(img)/255	    #0-1に変換
    img_nad = img_nad[None, ...]	    #四次元配列に
    label=['background','flare']
    pred = model_3_10keV.predict(img_nad, batch_size=1, verbose=0)
    score = np.max(pred)
    pred_label = label[np.argmax(pred[0])]
    val = 0.99
    if score > val and pred_label == "flare":
        a=jpeg_name.split("_")
        list=[int(a[0]),jpeg_name,score]     
        df.append(list)
result_3_10keV = pd.DataFrame(df)
result_3_10keV = result_3_10keV.sort_values([0])
result_3_10keV = result_3_10keV.reset_index(drop=True)
result_3_10keV = result_3_10keV.iloc[:,0:3]


KeyError: 0

In [37]:
pd.DataFrame(df).empty

True